# 爬取虎扑的赛后评论，并分析

In [1]:
import requests
import re
from collections import Counter
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib.font_manager import _rebuild
_rebuild()

## 找到赛后链接

In [2]:
def order_win_lose(title):
    pattern = '\[赛后\](.*?)(\d+)-(\d+)(.*?)，'
    game_result = re.findall(pattern,title,re.S)[0]
    team_1 = game_result[0]
    team_2 = game_result[3].replace('不敌','')
    if float(game_result[1])>float(game_result[2]):
        return (team_1,team_2)
    else:
        return (team_2,team_1)

In [3]:
order_win_lose('[赛后]公牛86-118不敌火箭，马尔卡宁22+4，戈登31分，保罗13+10，火箭十连胜')

('火箭', '公牛')

In [4]:
# url:'https://bbs.hupu.com/vote'
# sample target: ('https://bbs.hupu.com/21797416.html',
# '[赛后]公牛86-118火箭，马尔卡宁22+4，戈登31分，保罗13+10，火箭十连胜',
# '20')
def get_after_game_post(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'lxml')
    lis = soup.find_all('li')
    targets = []
    for li in lis:
        target = li.find('a',class_='truetit')
        if target!=None and target.text.find('[赛后]')!=-1:
            page_n = li.find('span',class_='multipage').find_all('a')[-1].text 
            targets.append(('https://bbs.hupu.com'+target['href'],order_win_lose(target.text),page_n))
    return targets

## 统计评论人主队

In [5]:
def counter_fans(post_url,page_n):
    counter = Counter()
    i=1
    while i<=page_n:
        try:
            after_game_url =post_url.format(i)
            response = requests.get(after_game_url)
            # <div uname="疯狂的麦克斯4" uid="278913245705307" class="j_u">
            #<a href="https://my.hupu.com/278913245705307" target="_blank" class="headpic"><img src="https://i3.hoopchina.com.cn/user/header/20170415/149222410134740_small_1.jpg" width="45" height="45" alt="疯狂的麦克斯4"></a>
            #</div>
            results = re.findall('<a\sclass="u"\starget="_blank"\shref="https://my.hupu.com/(.*?)".*?>',response.text,re.S)
            fans = []
            for result in results:
                usernum = result
                url = 'https://my.hupu.com/{}'.format(usernum)
                resp = requests.get(url)
                pat = re.compile('<span.*?>NBA主队.*?<a.*?>(.*?)</a>.*?',re.S)
                team = re.findall(pat,resp.text)
                if len(team)>0:
                    fans.append(team[0])
                else:
                    fans.append('无主队')
            counter.update(fans)
            print('in {} page {}'.format(post_url,i))
            i+=1
        except Exception:
            print('end')
    return counter

## 把帖子信息更新到文件中

In [6]:
url = 'https://bbs.hupu.com/vote'
post_infos = get_after_game_post(url)

In [7]:
post_infos_df = pd.DataFrame(post_infos)
post_infos_df.columns=['address','teams','page']
with open('post_infos.csv','w') as f:
    post_infos_df.to_csv(f)

In [8]:
old_post_infos = pd.read_csv('post_infos.csv',usecols=['address','teams','page'])
old_post_infos.head(1)

,address,teams,page
0,https://bbs.hupu.com/21816857.html,"('雄鹿', '勇士')",18


In [9]:
old_post_url = set(old_post_infos['address'])
results = []
new_post_infos = []
for i in range(1,11):
    url = 'https://bbs.hupu.com/vote-{}'.format(i)
    new_post_infos.extend(get_after_game_post(url))
    print('for loop:{}'.format(i))
new_post_infos

for loop:1
for loop:2
for loop:3
for loop:4
for loop:5
for loop:6
for loop:7
for loop:8
for loop:9
for loop:10


[('https://bbs.hupu.com/21816857.html', ('雄鹿', '勇士'), '18'),
 ('https://bbs.hupu.com/21815678.html', ('马刺', '雷霆'), '55'),
 ('https://bbs.hupu.com/21816663.html', ('步行者', '国王'), '5'),
 ('https://bbs.hupu.com/21815273.html', ('热火', '公牛'), '2'),
 ('https://bbs.hupu.com/21815004.html', ('活塞', '奇才'), '6'),
 ('https://bbs.hupu.com/21807714.html', ('凯尔特人', '爵士'), '20'),
 ('https://bbs.hupu.com/21730022.html', ('马刺', '勇士'), '32'),
 ('https://bbs.hupu.com/21789432.html', ('灰熊', '森林狼'), '28'),
 ('https://bbs.hupu.com/21745462.html', ('黄蜂', '篮网'), '25'),
 ('https://bbs.hupu.com/21805997.html', ('骑士', '黄蜂'), '24'),
 ('https://bbs.hupu.com/21806091.html', ('76人', '尼克斯'), '9'),
 ('https://bbs.hupu.com/21808310.html', ('湖人', '独行侠'), '15'),
 ('https://bbs.hupu.com/21806758.html', ('灰熊', '开拓者'), '16'),
 ('https://bbs.hupu.com/21799137.html', ('步行者', '勇士'), '10'),
 ('https://bbs.hupu.com/21805964.html', ('篮网', '魔术'), '5'),
 ('https://bbs.hupu.com/21806695.html', ('森林狼', '老鹰'), '12'),
 ('https://bbs.hupu

In [10]:
new_post_infos = pd.DataFrame(new_post_infos)
new_post_infos.columns=['address','teams','page']
new_post_url = set(new_post_infos['address'])
update_post_url = new_post_url-old_post_url
post_infos = old_post_infos.append(new_post_infos).drop_duplicates()

In [11]:
new_post_infos

,address,teams,page
0,https://bbs.hupu.com/21816857.html,"(雄鹿, 勇士)",18
1,https://bbs.hupu.com/21815678.html,"(马刺, 雷霆)",55
2,https://bbs.hupu.com/21816663.html,"(步行者, 国王)",5
3,https://bbs.hupu.com/21815273.html,"(热火, 公牛)",2
4,https://bbs.hupu.com/21815004.html,"(活塞, 奇才)",6
5,https://bbs.hupu.com/21807714.html,"(凯尔特人, 爵士)",20
6,https://bbs.hupu.com/21730022.html,"(马刺, 勇士)",32
7,https://bbs.hupu.com/21789432.html,"(灰熊, 森林狼)",28
8,https://bbs.hupu.com/21745462.html,"(黄蜂, 篮网)",25
9,https://bbs.hupu.com/21805997.html,"(骑士, 黄蜂)",24


In [ ]:
with open('post_infos.csv','w') as f:
    post_infos.to_csv(f)
    
for url in update_post_url:
    post_url = url
    page_n = int(new_post_infos[new_post_infos['address']==url]['page'])
    win_loss = new_post_infos[new_post_infos['address']==url]['teams']
    win = str(win_loss).replace('(','').replace(')','').split(',')[0]
    loss = str(win_loss).replace('(','').replace(')','').split(',')[1]
    counter = counter_fans(post_url,page_n)
    results.append((win,loss,counter))
print('complete {}/10'.format(i))

in https://bbs.hupu.com/21808310.html page 1


In [ ]:
result_df = pd.DataFrame(results)
result_df.columns = ['win','loss','count']

In [ ]:
print(len(result_df))
result_df.head()

In [ ]:
count_table = pd.DataFrame()
for i in range(len(result_df)):
    s1 = pd.Series(result_df['count'][i])
    count_table = pd.concat([count_table,s1],axis=1)

In [ ]:
count_table = count_table.T
count_table.reset_index(inplace=True)

In [ ]:
final_table = pd.concat([result_df[['win','loss']],count_table],axis=1)
final_table = final_table.drop('index',axis=1)
final_table = final_table.fillna(0)

In [ ]:
old_table = pd.read_csv('result_table.csv',usecols=[i for i in range(1,final_table.shape[1]+1)])
final_table = old_table.append(final_table).drop_duplicates()
with open('result_table.csv','w') as f:
    final_table.to_csv(f)

In [ ]:
np.sum(final_table.iloc[:,3:],axis=0).plot(kind='bar',figsize=(8,5),fontsize=12)